## Model building

In [2]:
import numpy as np
#set values of N and K 
N = 4
K = 2
X = 2**(K+1) #total fitness contributions per gene (= columns in fitness matrix)

Creating 3 matrices with versions of NK model:
- NK - without neutrality
- NKp - with neutrality (probabilistic)
- NKq - with neutrality (quantised)

*To-do:*
* *- fix fmp so that:*
    * *it is separate object*
    * *- p is per value in array*
* *code is shorter and one function?*
* *fm selection done at start of code?*

In [48]:
#create fitness matrix: np.array of shape (N,X), filled with random decimals 
fmk = np.random.rand(N, X)

#NKp: reduce fraction of fitness contributions in fm to 0
fmp = fmk.copy()
p = ((N-1)/N) #is it? :)
fmp[np.random.rand(*fmp.shape) < p] = 0

#NKq: fitness contribution < 0.5 == 0, > 0.5 == 1
fmq0 = fmk.copy()
fmq1 = np.where(fmq0 > 0.5, 1.0, fmq0)
fmq = np.where(fmq1 < 0.5, 0.0, fmq1)



In [ ]:
#old fmp code, remove when new works

#NKp: reduce fraction of fitness contributions in fm to 0
#Number of elements to replace
#p = ((N-1)/N)   
#num_p = int(((N-1)/N) * (N*X)) #(with p = (N-1)/N (Geard 10.1109/CEC.2002.1006234)) ##Make choosing p easy and more neat
#Random (x, y) coordinates
#indices_x = np.random.randint(0, fmk.shape[0], num_p)
#indices_y = np.random.randint(0, fmk.shape[1], num_p)
#fmp = fmk[indices_x, indices_y] = 0 # fix so 1) fmp is separate object from fmk (rn only works if fmp=>fm), 2) p is per value
#fmp = fmk.copy() #?

#check: below can be removed in final code
#print("fitness matrix without neutrality (fmk)")
#print(fmk) #no neutrality
#print("fitness matrix probabilistic NK (fmp)")
#print(fmp) #probabilistic NK
#print("fitness matrix quantised NK (fmq)")
#print(fmq) # = quantised NK

Choose neutrality version:

In [4]:
#replace fmk in following line with "fmp" or "fmq" when adding probabilistic or quantised neutrality to model
fm = np.copy(fmk) 
#makes deep copies (as does fmp/fmq before) maybe not necessary?

Creating corresponding epistasis matrix

In [5]:
#to do

#creates "identity matrix": array with genomes
im0 = np.arange(0, X, 1)
im1 = im0[np.newaxis, :]
im = np.repeat(im1, N, axis=0)
print(im)

#Binary representation of im (just for visualisation)
imbin = np.vectorize(np.binary_repr)(im, 4) #increase to 8/16/32 with larger N 
print("identity matrix")
print(imbin)
#check if shape is same
print(imbin.shape)

em0 = np.arange(0, N, 1)
em = np.random.rand(N, 1)

x = np.random.randint(1, 2, (N,K))


[[0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]]
identity matrix
[['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']
 ['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']
 ['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']
 ['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']]
(4, 8)


In [50]:
#important: in this version each gene influenced by K others, but genes 
# can influence >2 other genes, so some are (way) more influential than others
# print a few times to see
import random
val = list(range(1, N+1))
em1 = []
for row in range(1, N+1):
    val.remove(row)
    em1.append([random.choice(val), random.choice(val)])
    val.append(row)
#print(em1)
em1 = np.array(em1)
print(em1)


[[4 3]
 [1 3]
 [2 2]
 [2 2]]


In [46]:
#important: in this version each gene influenced by K others, 
# and influences K others, so all equally influential
# print a few times to see

import random

def generate_all_moved_permutation_tree(level, nums):
    if len(nums) == 0:
        raise RunTimeError('generate_permutation_tree must be called with a non-empty nums list')
    if len(nums) == 1:
        if level == nums[0]:
            return None
        else:
            return {nums[0]: {}}
    allowed_nums = list(nums)
    if level in allowed_nums:
        allowed_nums.remove(level)
    result = {}
    for n in allowed_nums:
        sublevel_nums = list(nums)
        if n in sublevel_nums:
            sublevel_nums.remove(n)
        subtree = generate_all_moved_permutation_tree(level+1, sublevel_nums)
        if subtree is not None:
            result[n] = subtree
    if len(result) == 0:
        return None
    return result

def pick_an_all_moved_permutation(all_moved_permutation_tree):
    n = random.choice(list(all_moved_permutation_tree))
    l = [n]
    sub_tree = all_moved_permutation_tree[n]
    if len(sub_tree) > 0:
        l.extend(pick_an_all_moved_permutation(sub_tree))
    return l

t = generate_all_moved_permutation_tree(1, range(1,5))
em2 = list(zip(pick_an_all_moved_permutation(t), pick_an_all_moved_permutation(t)))
em2 = np.array(em2)

print(em1) #for comparison with and without replacement
print(em2)

[[2 4]
 [1 4]
 [2 2]
 [2 2]]
[[4 3]
 [1 4]
 [2 1]
 [3 2]]


In [ ]:
#first em tries that didn't work (remove from final code):
#np.random.shuffle(em0[:,0])
#np.random.shuffle(em0[:,1])
#print(em0) 
#this does not work because it does not exclude row value itself

#for col in range(em.shape[1]): 
    #np.random.shuffle(em[:,col])
#if np.setdiff1d(em, em0, em0.size > 0) == N*K: 
#this does not work because it loops per row

Calculating coefficients ai0 to aij

*To-do*
* *annotate more clearly*


In [47]:
def calc_a(K, fm): 
    a_coef = []
    #explain for loop
    for r in fm:
        a = [0.0] * X  # creates list with zeros as floats for each row & X cols
        
        # Calculate ai0 for i = 0
        a[0] = r[0] #because ai0=Fi0
        
        for j in range(1, X): 
            sum = 0.0 
            for l in range(0, j): #only already calculated coeff
                if l == (l & j): #if l equal to bitwise AND of l and j (001&101->001 so TRUE, 001&100->000 so FALSE)
                    sum += a[l] 
            a[j] = r[j] - sum 
        a_coef.append(a) # append new a's into a_values array
        
    return a_coef

a_coef = calc_a(K, fm)

a_shape = np.reshape(a_coef, (N,X))

if fm.all == fmk.all: 
    print("Coefficient matrix fmk")
elif fm.all == fmp.all:
    print("Coefficient matrix fmp")
else: 
    print("Coefficient matrix fmq")
print(a_shape) #all coefficients in same shape as fm1

Coefficient matrix fmq
[[ 0.24205212  0.66380135 -0.19061641 -0.30240139 -0.13748196  0.12530041
   0.72569477 -0.91148981]
 [ 0.59453673  0.11451389 -0.03320204 -0.63863587  0.03701525 -0.55839875
  -0.01673426  0.56624105]
 [ 0.92951227 -0.86693889 -0.17657679  1.05578942 -0.9123321   1.81106915
   1.0186448  -2.29409743]
 [ 0.93783871 -0.70550248 -0.85808994  1.16141227 -0.04718263  0.80537302
   0.44645969 -1.55478013]]


Compute model for genome fitness

In [ ]:
#To-do